<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/GPT_2/GPT2_GEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentencepiece : https://github.com/google/sentencepiece/blob/master/python/README.md

### NSMC

In [1]:
import urllib

In [2]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt','train.txt')

('train.txt', <http.client.HTTPMessage at 0x7ab0c9cbda20>)

In [3]:
temp=[]

In [4]:
with open('train.txt','r') as f:
  f.readline()
  for sent in f.readlines():
    data = sent.split('\t')
    temp.append(data[1])

In [1]:
cd drive/MyDrive/Pytorch\ NLP/GPT-2/data_in/gpt2_ckpt

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in/gpt2_ckpt


In [ ]:
with open('corpus.txt','w') as f:
  for sent in temp:
    f.write(f'\n {sent}')

### Sentencepiece

https://keep-steady.tistory.com/7

In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00


In [3]:
import sentencepiece as spm

In [ ]:
 spm.SentencePieceTrainer.train(input='corpus.txt',model_prefix='kor_tokenizer',vocab_size=50000, model_type='bpe',max_sentence_length=9999999,pad_id=0,unk_id=1,bos_id=2,eos_id=3)

### gluonnlp

In [4]:
sp=spm.SentencePieceProcessor()
sp.Load('kor_tokenizer.model')

True

In [5]:
sp.EncodeAsPieces('안녕하세요 저는 사람이에요.')

['▁안녕하세요', '▁저는', '▁사람이', '에요', '.']

In [6]:
sp.encode('안녕하세요 저는 사람이에요.')

[20363, 1619, 1047, 802, 48289]

In [22]:
vocab = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]

In [27]:
print('0 :{}, 1 :{} , 2 :{}, 3 :{} '.format(vocab[0],vocab[1],vocab[2],vocab[3]))

0 :<pad>, 1 :<unk> , 2 :<s>, 3 :</s> 


### config & model_weight

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/config.json','config.json')

('config.json', <http.client.HTTPMessage at 0x7fdd07c1ada0>)

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/model.safetensors','model.safetensors')

('model.safetensors', <http.client.HTTPMessage at 0x7fdd07c19ed0>)

## Top k & Top p

### torch.multinomial

In [5]:
import torch

In [70]:
weights= torch.Tensor([0,10,5,5])
weigths=torch.nn.Softmax()(weights)
torch.multinomial(weigths,10,replacement=True)

<ipython-input-70-63ac0e2f0888>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weigths=torch.nn.Softmax()(weights)


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
  # logtis = (vocab_size, )

  if top_k>0:
    top_k=min(top_k,logits.size()[-1])
    indices_to_remove=logits<torch.topk(logits,top_k).values[-1] # smaller than the smallest value among the top_k values
    logits[indices_to_remove]=filter_value

  if top_p>0.0:

